In [24]:
%%timeit

import pandas as pd

df = pd.read_excel('./movies.xlsx', skiprows=7, usecols=[2, 3] )

df = df[df.genres != '(no genres listed)']

# Create new Dataframe
def extract_values(row):
    genres = row['genres'].split('|')
    movie = row['movie']
    return [[genre, movie] for genre in genres]

data = df.apply(extract_values, axis=1)

df_new = pd.DataFrame.from_records([y for x in data for y in x], columns=['genres', 'movie'])

df_new.groupby('genres').count().sort_values('movie', ascending=False)

125 ms ± 5.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
%%timeit

import pandas as pd
import numpy as np

movie_excel_file = "./movies.xlsx"


def explode(df, lst_cols, fill_value='', preserve_index=False):
    """Helper found on SO to split pipe (|) separted genres into
       multiple rows so it becomes easier to group the data -
       https://stackoverflow.com/a/40449726
    """
    if(lst_cols is not None and len(lst_cols) > 0 and not
       isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    idx_cols = df.columns.difference(lst_cols)
    lens = df[lst_cols[0]].str.len()
    idx = np.repeat(df.index.values, lens)
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    if (lens == 0).any():
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    res = res.sort_index()
    if not preserve_index:
        res = res.reset_index(drop=True)
    return res


def group_by_genre(data=movie_excel_file):
    """Takes movies data excel file (https://bit.ly/2BXra4w) and loads it
       into a DataFrame (df).

       Explode genre1|genre2|genre3 into separte rows using the provided
       "explode" function we found here: https://bit.ly/2Udfkdt

       Filters out '(no genres listed)' and groups the df by genre
       counting the movies in each genre.

       Return the new df of shape (rows, cols) = (19, 1) sorted by movie count
       descending (example output: https://bit.ly/2ILODva)
    """
    movies = pd.read_excel(movie_excel_file, skiprows=7, usecols=[2, 3])
    movies.genres = movies.genres.str.split('|')
    movies = explode(movies, ['genres'])
    movies = movies[movies['genres'] != '(no genres listed)']
    grouped = movies.groupby(['genres']).count().sort_values(by="movie")
    return grouped.sort_values('movie', ascending=False)

group_by_genre()

42.3 ms ± 2.65 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
